# Простые алгоритмы

Из [этого курса на Stepik](https://stepik.org/course/8057/syllabus)

## init

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

In [3]:
plt.rcParams["figure.figsize"] = (17,8)
plt.style.use('ggplot')

np.random.seed(42)

## расстояния

Дополнительная темка  

https://docs.scipy.org/doc/scipy/reference/spatial.distance.html  

https://ml-handbook.ru/chapters/metric_based/intro#%D0%B2%D1%8B%D0%B1%D0%BE%D1%80-%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D0%BA%D0%B8  

**Свойства метрики:**  
* $\rho(A, A) = 0$: расстояние до самого себя равно 0;  
* $\rho(A, B) = \rho(B, A)$: метрика симметрична;  
* $\rho(A, B) \leq \rho(A, C) + \rho(C, B)$: метрика удовлетворяет **неравенству треугольника** - расстояние между любыми двумя точками не может быть больше, чем проход между ними через промежуточный пункт (третью точку);  
* $\rho(A, B) \in (0, +\inf)$: метрика неотрицательна.

> _Не все приведенные ниже "расстояния" удовлетворяют неравенству треугольника, т.е. не являются **метриками** в строгом математическом смысле. Но все неотрицательны и симметричны_

In [4]:
from scipy.spatial import distance

In [5]:
a = np.array([1,2,3])
b = np.array([2,1,4])

In [38]:
# Euclidean
temp = a - b

dist0 = np.sqrt(np.sum(np.square(temp)))
dist1 = np.linalg.norm(temp)
dist2 = np.sqrt(np.dot(temp.T, temp))
dist3 = distance.euclidean(a, b)

dist0, dist1, dist2, dist3

(1.7320508075688772,
 1.7320508075688772,
 1.7320508075688772,
 1.7320508075688772)

In [39]:
# Manhattan (City Block)
temp = a - b

dist0 = np.sum(np.abs(temp))
dist1 = distance.cityblock(a, b)

dist0, dist1

(3, 3)

Менее чувствительно к выбросам в многомерном пространстве. Например, если у нас 1000 признаков, и по 999 два объекта близки, а по 1000-му - отличаются заметнее, евклидово расстояние возведет в квадрат, усилит эту разницу, а манхэттен - нет.

In [40]:
# Cosine distance
dist0 = 1 - ( np.dot(a.T, b) / ( np.sqrt(np.sum(np.square(a))) * np.sqrt(np.sum(np.square(b))) ) )
dist1 = 1 - (np.dot(a.T, b) / ( np.linalg.norm(a) * np.linalg.norm(b) ) )
dist2 = distance.cosine(a, b)

dist0, dist1, dist2

(0.06686105036831302, 0.06686105036831302, 0.06686105036831325)

Любят в NLP. Потому что не зависит от нормы (длины вектора). Представим, что у нас bag of words, и мы тупо считаем кол-во вхождений. Вот если у нас тех же слов, но не по одному, а по 2 встречается, тема же от этого не меняется. А норма меняется - в два раза больше становится. Поэтому и юзается расстояние, не зависящее от нормы, длины вектора. Только его направление нас интересует.  

Но кажется тут с правилом треугольника проблемы...

**Кстати!**  

$ \frac{u \cdot v}{||u||_2 * ||v||_2} $ называют _cosine similarity_ между векторами $u$ и $v$, то есть _косинусное сходство_. Здесь $u \cdot v$ - это скалярное произведение векторов $u$ и $v$, а $||u||_2$ - это $L_2$-норма, она же Евклидова норма, она же длина вектора $u$.  

Так вот:  
$ cosine\_distance = 1 - cosine\_similarity $  
$ cosine\_similarity = 1 - cosine\_distance $  

А вот Манхеттенское расстояние - это к вопросу об $L_1$-норме: сумма модулей. А $L_2$-норма, она же Евклидова - это про корень из суммы квадратов.

In [10]:
# Minkowski distance

p = 3

dist0 = np.sum(
    np.abs(a - b)**p
)**(1/p)
dist1 = distance.minkowski(a, b, p)

dist0, dist1

(1.4422495703074083, 1.4422495703074083)

Метрика Минковского - это обобщение евклидовой ($p=2$) и манхэттенской ($p=1$) метрик. В данном случае $p$ - параметр.  
Формула, для наглядности:  
$$\rho(u, v, p) = (\sum_i |u_i - v_i|^p )^\frac{1}{p}$$

In [11]:
# for bools:
# Jaccard-Needham dissimilarity
a = np.array([True, False, True, False])
b = np.array([False, True, True, False])

dist0 = 1 - np.sum(a & b) / np.sum(a | b)
dist1 = distance.jaccard(a, b)

dist0, dist1

(0.6666666666666667, 0.6666666666666666)

Расстояние Жаккара, также как и косинусное расстояние, может легко превратиться в "близость": достаточно убрать `1-`. Если в терминах множеств говорить (а булев массив как раз может кодировать множества: входит в него элемент, или нет), получится так:  
$$\rho(U, V) = 1 - \frac{|U \cap V|}{|U \cup V|}$$  

Ну и с правилом треугольника тут тоже всё так себе, похоже...

## восстановление данных/рекомендации

In [47]:
shape = (10, 5)

data = pd.DataFrame(
    np.random.normal(loc=100, scale=20, size=shape)
)

data

,0,1,2,3,4
0,105.009857,106.928964,86.399506,104.645074,105.861449
1,85.712972,137.315490,109.476658,76.173930,113.131072
2,80.506367,115.741692,123.171912,83.586354,119.267523
3,108.255619,116.441203,137.935860,95.092238,84.925277
4,82.209711,83.683794,98.457966,106.823039,105.533816
5,116.543665,100.260038,129.070682,94.706863,154.403383
6,112.513347,82.856849,78.582150,109.649448,95.530744
7,114.280010,109.464752,98.543422,83.064126,69.703056
8,91.069701,117.127976,104.281875,75.085224,103.463619
9,107.706348,82.322851,103.074502,101.164174,77.140594


Сделаем пропуск в колонке **0**:  

In [48]:
data.at[0, 0] = np.nan

data

,0,1,2,3,4
0,NaN,106.928964,86.399506,104.645074,105.861449
1,85.712972,137.315490,109.476658,76.173930,113.131072
2,80.506367,115.741692,123.171912,83.586354,119.267523
3,108.255619,116.441203,137.935860,95.092238,84.925277
4,82.209711,83.683794,98.457966,106.823039,105.533816
5,116.543665,100.260038,129.070682,94.706863,154.403383
6,112.513347,82.856849,78.582150,109.649448,95.530744
7,114.280010,109.464752,98.543422,83.064126,69.703056
8,91.069701,117.127976,104.281875,75.085224,103.463619
9,107.706348,82.322851,103.074502,101.164174,77.140594


Итак, задача: заполнить дырку. Варианты среднего и медианы не смотрим. Смотрим поинтереснее.

### Близость объектов (строк)

Считаем близость (по какой-либо метрике) нашего объекта (строки) к другим объектам по известным колонкам:  

In [55]:
nan_object = data.loc[0, 1:4]

nan_object

1    106.928964
2     86.399506
3    104.645074
4    105.861449
Name: 0, dtype: float64

In [56]:
distances = data.loc[1:, 1:4].apply(lambda x: distance.euclidean(x, nan_object),  axis=1)

distances

1    48.159624
2    45.310777
3    57.236863
4    26.279179
5    65.729683
6    27.791096
7    43.898425
8    36.101707
9    41.479157
dtype: float64

> тут нужно, чтобы значения были нормированы (шкалированы, отмасштабированы). Иначе расстояние посчитается криво.

Теперь смотрим, какие значения в интересующей нас колонке (признак **0**) у известных нам объектов:  

In [57]:
nan_column_values = data.loc[1:, 0]

nan_column_values

1     85.712972
2     80.506367
3    108.255619
4     82.209711
5    116.543665
6    112.513347
7    114.280010
8     91.069701
9    107.706348
Name: 0, dtype: float64

И вот теперь взвешиваем значения на расстояния (точнее, на близость - меру, обратную расстоянию), и получаем искомое значение:

In [72]:
similarity = 1 / distances

fill_value = np.sum(nan_column_values * similarity) * (1 / np.sum(similarity))

fill_value

98.6886027455759

`* (1 / np.sum(similarity))` - это для нормирования. Как вариант, можно `similarity` перевести в значения, дающие в сумме 1:  

In [73]:
norm_similarity = similarity / sum(similarity)

sum(norm_similarity)

1.0

In [74]:
fill_value = np.sum(nan_column_values * norm_similarity)

fill_value

98.68860274557589

In [75]:
np.dot(nan_column_values, norm_similarity) # ещё так можно прописать

98.68860274557588

### Корреляция признаков (колонок)

Берем значения нашего объекта по известным столбцам:

In [77]:
nan_object = data.loc[0, 1:4]

nan_object

1    106.928964
2     86.399506
3    104.645074
4    105.861449
Name: 0, dtype: float64

Берем матрицу без нашего объекта:

In [93]:
other_objects_matrix = data.loc[1:] # без строки nan_object

other_objects_matrix

,0,1,2,3,4
1,85.712972,137.315490,109.476658,76.173930,113.131072
2,80.506367,115.741692,123.171912,83.586354,119.267523
3,108.255619,116.441203,137.935860,95.092238,84.925277
4,82.209711,83.683794,98.457966,106.823039,105.533816
5,116.543665,100.260038,129.070682,94.706863,154.403383
6,112.513347,82.856849,78.582150,109.649448,95.530744
7,114.280010,109.464752,98.543422,83.064126,69.703056
8,91.069701,117.127976,104.281875,75.085224,103.463619
9,107.706348,82.322851,103.074502,101.164174,77.140594


На ней считаем средние по столбцам:

In [94]:
columns_means = other_objects_matrix.mean()[1:] # без колонки nan_column

columns_means

1    105.023850
2    109.177225
3     91.705044
4    102.566565
dtype: float64

In [96]:
nan_column_mean = other_objects_matrix[0].mean()

nan_column_mean

99.86641539469383

И корреляцию столбца с неизвестным VS все остальные столбцы:

In [95]:
nan_column_corr = other_objects_matrix.corrwith(other_objects_matrix[0])[1:] 
# без колонки nan_column (без корреляции с самой собой)

nan_column_corr

1   -0.341277
2   -0.026963
3    0.337373
4   -0.192735
dtype: float64

Тут мы рассматриваем отклонение значений `nan_object` от средних, и взвешиваем это дело на корреляцию (со столбцом, в котором неизвестное значение). И получившееся "отклонение" добавляем к среднему значению по столбцу с неизвестным:

In [98]:
fill_value = \
    nan_column_mean + (
        np.sum(nan_column_corr * (nan_object - columns_means)) / np.sum(np.abs(nan_column_corr))
    )

fill_value

103.97903293857124

Тут опять `np.sum(np.abs(nan_column_corr))` - элемент нормализации. Можно заранее нормализовать веса:  

In [101]:
weights = nan_column_corr / np.sum(np.abs(nan_column_corr))

fill_value = nan_column_mean + np.sum(weights * (nan_object - columns_means))

fill_value

103.97903293857124

In [102]:
# или даже так:
nan_column_mean + np.dot(weights, (nan_object - columns_means))

103.97903293857124

In [109]:
df = pd.DataFrame({
    'a': ['a', 'a', 'b', 'b'],
    'b': [1,2,3,4]
})

lol = 'b'

df.query('a == @lol')

,a,b
2,b,3
3,b,4
